In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from SimpleCNN18K import SimpleCNN18K

ImportError: cannot import name 'simpleCNN18K' from 'SimpleCNN18K' (d:\Projects\ERAV3\Session6\SimpleCNN18K.py)

Assign the Device

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else 'cpu')
print(device)

cuda


Download the data

In [ ]:
torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers':1,'pin_memory':True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=True,download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,),(0.3081,))]
                   )),batch_size=batch_size,shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=False,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,),(0.3081,))
                                                 ])),batch_size=batch_size,shuffle=True,**kwargs)

Train and Test Functions

In [ ]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  train_loss = 0
  correct = 0
  pbar = tqdm(train_loader)
  for batch_idx, (data,target) in enumerate(pbar):
    data,target = data.to(device),target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output,target)
    loss.backward()
    optimizer.step()
    pbar.set_description(desc=f'loss={loss.item()} batch_id={batch_idx}')
    with torch.no_grad():
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
  train_loss /= len(train_loader.dataset)
  print('\n Epoch No: {}'.format(epoch))
  print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
    train_loss, correct, len(train_loader.dataset),
    100. * float(correct) / len(train_loader.dataset)))

def test(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data,target in test_loader:
      data,target = data.to(device),target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output,target,reduction='sum').item()
      pred = output.argmax(dim=1,keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print('Test set: Average loss: {:.4f}, Accuracy:{}/{} ({:.2f}%)\n'.format(
      test_loss,correct,len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

Define the CNN model with different archicture and check for expected results

Version 1

In [ ]:
class Net_v1(nn.Module):
  def __init__(self):
    super(Net_v1,self).__init__()
    self.conv1 = nn.Conv2d(1,8,3,stride=1,padding=1) # input_size = 28, output_size = 28, RF = 3, No_param =
    self.conv2 = nn.Conv2d(8,8,3,stride=1,padding=1)# input_size = 28, output_size = 28, RF = 5
    self.pool1 = nn.MaxPool2d(2,2)                    # input_size = 28, output_size = 14, RF = 6
    self.conv3 = nn.Conv2d(8,16,3,stride=1,padding=1)# input_size = 14, output_size = 14, RF = 10
    self.conv4 = nn.Conv2d(16,16,3,stride=1,padding=1)# input_size = 14, output_size = 14, RF = 14
    self.pool2 = nn.MaxPool2d(2,2)                      # input_size = 14, output_size = 7, RF = 16
    self.conv5 = nn.Conv2d(16,32,3,stride=1,padding=0)# input_size = 7, output_size = 5, RF = 24
    self.conv6 = nn.Conv2d(32,32,3,stride=1,padding=0)# input_size = 5, output_size = 3, RF = 32
    self.conv7 = nn.Conv2d(32,10,3)                    # input_size = 3, output_size = 1, RF = 40

  def forward(self,x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    x = self.conv7(x)
    x = x.view(-1,10)
    return F.log_softmax(x)

In [ ]:
model = Net_v1().to(device)
summary(model,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2            [-1, 8, 28, 28]             584
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
            Conv2d-5           [-1, 16, 14, 14]           2,320
         MaxPool2d-6             [-1, 16, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]           4,640
            Conv2d-8             [-1, 32, 3, 3]           9,248
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 20,930
Trainable params: 20,930
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.17
Params size (MB): 0.08
Estimated Total Size (MB): 0.25
---------------------------------------------

C:\Users\ic016726\AppData\Local\Temp\ipykernel_22012\948642335.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
model = Net_v1().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9)

for epoch in range(1,10):
  train(model,device, train_loader, optimizer, epoch)
  test(model,device, train_loader)

  0%|          | 0/3750 [00:00<?, ?it/s]C:\Users\ic016726\AppData\Local\Temp\ipykernel_22012\948642335.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.05615783482789993 batch_id=3749: 100%|██████████| 3750/3750 [00:22<00:00, 164.80it/s]   



 Epoch No: 1
Train set: Average loss: 0.0000, Accuracy: 56021/60000 (93.37%)
Test set: Average loss: 0.0678, Accuracy:58737/60000 (97.89%)



loss=0.008126807399094105 batch_id=3749: 100%|██████████| 3750/3750 [00:23<00:00, 160.57it/s]  



 Epoch No: 2
Train set: Average loss: 0.0000, Accuracy: 59208/60000 (98.68%)
Test set: Average loss: 0.0508, Accuracy:59074/60000 (98.46%)



loss=0.00045628711814060807 batch_id=3749: 100%|██████████| 3750/3750 [00:23<00:00, 157.68it/s]

...


 Epoch No: 7
Train set: Average loss: 0.0000, Accuracy: 59680/60000 (99.47%)
Test set: Average loss: 0.0239, Accuracy:59527/60000 (99.21%)



loss=0.00017353170551359653 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 143.57it/s]



 Epoch No: 8
Train set: Average loss: 0.0000, Accuracy: 59769/60000 (99.61%)
Test set: Average loss: 0.0216, Accuracy:59572/60000 (99.29%)



loss=0.029669830575585365 batch_id=3749: 100%|██████████| 3750/3750 [00:23<00:00, 159.21it/s]  



 Epoch No: 9
Train set: Average loss: 0.0000, Accuracy: 59784/60000 (99.64%)
Test set: Average loss: 0.0358, Accuracy:59303/60000 (98.84%)



Version 2

In [ ]:
class Net_v2(nn.Module):
  def __init__(self):
    super(Net_v2,self).__init__()
    self.conv1 = nn.Conv2d(1,8,3,stride=1,padding=1) # input_size = 28, output_size = 28, RF = 3,
    self.conv1_bn = nn.BatchNorm2d(8)
    self.conv2 = nn.Conv2d(8,16,3,stride=1,padding=0)# input_size = 28, output_size = 26, RF = 5
    self.conv2_bn = nn.BatchNorm2d(16)
    self.pool1 = nn.MaxPool2d(2,2)                    # input_size = 26, output_size = 13, RF = 6
    self.conv3 = nn.Conv2d(16,16,3,stride=1,padding=1)# input_size = 13, output_size = 13, RF = 10
    self.conv3_bn = nn.BatchNorm2d(16)
    self.conv4 = nn.Conv2d(16,32,3,stride=1,padding=0)# input_size = 13, output_size = 11, RF = 14
    self.conv4_bn = nn.BatchNorm2d(32)
    self.pool2 = nn.MaxPool2d(2,2)                      # input_size = 11, output_size = 5, RF = 16
    self.conv5 = nn.Conv2d(32,32,3,stride=1,padding=0)# input_size = 5, output_size = 3, RF = 24
    self.conv5_bn = nn.BatchNorm2d(32)
    self.pool3 = nn.AdaptiveAvgPool2d((1, 1))  # reduces the spatial dimensions to 1x1
    self.conv6 = nn.Conv2d(32,10,1)                   # input_size = 3, output_size = 1, RF = 40

  def forward(self,x):
    x = self.pool1(F.relu(self.conv2_bn(self.conv2(F.relu(self.conv1_bn(self.conv1(x)))))))
    x = self.pool2(F.relu(self.conv4_bn(self.conv4(F.relu(self.conv3_bn(self.conv3(x)))))))
    x = self.pool3(F.relu(self.conv5_bn(self.conv5(x))))
    x = self.conv6(x)
    x = x.view(-1,10)
    return F.log_softmax(x)

In [ ]:
model_2 = Net_v2().to(device)
summary(model_2,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 26, 26]           1,168
       BatchNorm2d-4           [-1, 16, 26, 26]              32
         MaxPool2d-5           [-1, 16, 13, 13]               0
            Conv2d-6           [-1, 16, 13, 13]           2,320
       BatchNorm2d-7           [-1, 16, 13, 13]              32
            Conv2d-8           [-1, 32, 11, 11]           4,640
       BatchNorm2d-9           [-1, 32, 11, 11]              64
        MaxPool2d-10             [-1, 32, 5, 5]               0
           Conv2d-11             [-1, 32, 3, 3]           9,248
      BatchNorm2d-12             [-1, 32, 3, 3]              64
AdaptiveAvgPool2d-13             [-1, 32, 1, 1]               0
           Conv2d-14             [-1, 1

C:\Users\ic016726\AppData\Local\Temp\ipykernel_22012\3380533065.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
optimizer = optim.SGD(model_2.parameters(),lr=0.01,momentum=0.9)

for epoch in range(1,10):
  train(model_2,device, train_loader, optimizer,epoch)
  test(model_2,device, train_loader)

  0%|          | 0/3750 [00:00<?, ?it/s]C:\Users\ic016726\AppData\Local\Temp\ipykernel_22012\3380533065.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.008845629170536995 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 144.10it/s] 



 Epoch No: 1
Train set: Average loss: 0.0000, Accuracy: 58278/60000 (97.13%)
Test set: Average loss: 0.0401, Accuracy:59268/60000 (98.78%)



loss=0.004290332086384296 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 139.83it/s]  



 Epoch No: 2
Train set: Average loss: 0.0000, Accuracy: 59337/60000 (98.89%)
Test set: Average loss: 0.0259, Accuracy:59547/60000 (99.25%)



loss=0.0009539314778521657 batch_id=3749: 100%|██████████| 3750/3750 [00:27<00:00, 138.09it/s] 


...


 Epoch No: 7
Train set: Average loss: 0.0000, Accuracy: 59716/60000 (99.53%)
Test set: Average loss: 0.0145, Accuracy:59722/60000 (99.54%)



loss=0.004844012204557657 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 144.12it/s]  



 Epoch No: 8
Train set: Average loss: 0.0000, Accuracy: 59753/60000 (99.59%)
Test set: Average loss: 0.0098, Accuracy:59822/60000 (99.70%)



loss=0.00948680005967617 batch_id=3749: 100%|██████████| 3750/3750 [00:26<00:00, 143.64it/s]   



 Epoch No: 9
Train set: Average loss: 0.0000, Accuracy: 59793/60000 (99.66%)
Test set: Average loss: 0.0068, Accuracy:59878/60000 (99.80%)



In [ ]:
class Net_v3(nn.Module):
  def __init__(self):
    super(Net_v3,self).__init__()
    self.conv1 = nn.Conv2d(1,8,3,stride=1,padding=1) # input_size = 28, output_size = 28, RF = 3,
    self.conv1_bn = nn.BatchNorm2d(8)
    self.conv2 = nn.Conv2d(8,16,3,stride=1,padding=0)# input_size = 28, output_size = 26, RF = 5
    self.conv2_bn = nn.BatchNorm2d(16)
    self.drop1 = nn.Dropout2d(0.05)
    self.pool1 = nn.MaxPool2d(2,2)                    # input_size = 26, output_size = 13, RF = 6
    self.conv3 = nn.Conv2d(16,16,3,stride=1,padding=1)# input_size = 13, output_size = 13, RF = 10
    self.conv3_bn = nn.BatchNorm2d(16)
    self.drop2 = nn.Dropout2d(0.05)
    self.conv4 = nn.Conv2d(16,32,3,stride=1,padding=0)# input_size = 13, output_size = 11, RF = 14
    self.conv4_bn = nn.BatchNorm2d(32)
    self.drop3 = nn.Dropout2d(0.05)
    self.pool2 = nn.MaxPool2d(2,2)                      # input_size = 11, output_size = 5, RF = 16
    self.conv5 = nn.Conv2d(32,32,3,stride=1,padding=0)# input_size = 5, output_size = 3, RF = 24
    self.conv5_bn = nn.BatchNorm2d(32)
    self.pool3 = nn.AdaptiveAvgPool2d((1, 1))  # reduces the spatial dimensions to 1x1
    self.conv6 = nn.Conv2d(32,10,1)                   # input_size = 3, output_size = 1, RF = 40

  def forward(self,x):
    x = self.pool1(F.relu(self.drop1(self.conv2_bn(self.conv2(F.relu(self.conv1_bn(self.conv1(x))))))))
    x = self.pool2(F.relu(self.drop3(self.conv4_bn(self.conv4(F.relu(self.drop2(self.conv3_bn(self.conv3(x)))))))))
    x = self.pool3(F.relu(self.conv5_bn(self.conv5(x))))
    x = self.conv6(x)
    x = x.view(-1,10)
    return F.log_softmax(x)


In [ ]:
model_3 = SimpleCNN18K().to(device)
summary(model_3,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 26, 26]           1,168
       BatchNorm2d-4           [-1, 16, 26, 26]              32
         Dropout2d-5           [-1, 16, 26, 26]               0
         MaxPool2d-6           [-1, 16, 13, 13]               0
            Conv2d-7           [-1, 16, 13, 13]           2,320
       BatchNorm2d-8           [-1, 16, 13, 13]              32
         Dropout2d-9           [-1, 16, 13, 13]               0
           Conv2d-10           [-1, 32, 11, 11]           4,640
      BatchNorm2d-11           [-1, 32, 11, 11]              64
        Dropout2d-12           [-1, 32, 11, 11]               0
        MaxPool2d-13             [-1, 32, 5, 5]               0
           Conv2d-14             [-1, 3

C:\Users\ic016726\AppData\Local\Temp\ipykernel_22012\1112118242.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
optimizer = optim.SGD(model_3.parameters(),lr=0.01,momentum=0.9)

for epoch in range(1,11):
  train(model_3,device, train_loader, optimizer,epoch)
  test(model_3,device, train_loader)

  0%|          | 0/3750 [00:00<?, ?it/s]C:\Users\ic016726\AppData\Local\Temp\ipykernel_22012\1112118242.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.007054571993649006 batch_id=3749: 100%|██████████| 3750/3750 [00:42<00:00, 88.74it/s]  



 Epoch No: 1
Train set: Average loss: 0.0000, Accuracy: 57110/60000 (95.18%)
Test set: Average loss: 0.0352, Accuracy:59355/60000 (98.92%)



loss=0.010384917259216309 batch_id=3749: 100%|██████████| 3750/3750 [00:42<00:00, 88.00it/s]  



 Epoch No: 2
Train set: Average loss: 0.0000, Accuracy: 58906/60000 (98.18%)
Test set: Average loss: 0.0279, Accuracy:59484/60000 (99.14%)



loss=0.006816477049142122 batch_id=3749: 100%|██████████| 3750/3750 [00:42<00:00, 88.56it/s]  


...


 Epoch No: 8
Train set: Average loss: 0.0000, Accuracy: 59449/60000 (99.08%)
Test set: Average loss: 0.0140, Accuracy:59753/60000 (99.59%)



loss=0.001169041614048183 batch_id=3749: 100%|██████████| 3750/3750 [00:41<00:00, 89.87it/s]   



 Epoch No: 9
Train set: Average loss: 0.0000, Accuracy: 59501/60000 (99.17%)
Test set: Average loss: 0.0129, Accuracy:59770/60000 (99.62%)



loss=0.005601446144282818 batch_id=3749: 100%|██████████| 3750/3750 [00:41<00:00, 90.39it/s]   



 Epoch No: 10
Train set: Average loss: 0.0000, Accuracy: 59534/60000 (99.22%)
Test set: Average loss: 0.0110, Accuracy:59796/60000 (99.66%)

